# TP5-6 : Problème (P1) - contraintes de précédence

Initialization (à décommenter et exécuter une seule fois)

In [8]:
#import Pkg; Pkg.add("Clp")

Récupération des données

In [2]:
D=[2, 3, 1, 4, 1]

5-element Array{Int64,1}:
 2
 3
 1
 4
 1

Modélisation et résolution de (P1) à l'aide d'un programme linéaire

In [3]:
using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:5] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1] )
@constraint(model, t[3] - t[1] >= D[1] )
@constraint(model, t[4] - t[2] >= D[2] )
@constraint(model, t[4] - t[3] >= D[3] )
@constraint(model, t[5] - t[3] >= D[3] )

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[4] >= D[4] )
@constraint(model, tfin - t[5] >= D[5] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))


Min tfin
Subject to
 -t[1] + t[2] ≥ 2.0
 -t[1] + t[3] ≥ 2.0
 -t[2] + t[4] ≥ 3.0
 -t[3] + t[4] ≥ 1.0
 -t[3] + t[5] ≥ 1.0
 -t[4] + tfin ≥ 4.0
 -t[5] + tfin ≥ 1.0
 t[1] ≥ 0.0
 t[2] ≥ 0.0
 t[3] ≥ 0.0
 t[4] ≥ 0.0
 t[5] ≥ 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0, 2.0, 2.0, 5.0, 8.0]	 tfin=9.0


## Modélisation classique par graphe potentiel-tache 

In [2]:
function PlusLongChemin(V, w, s)
    D = w[findall(x->x==s, V)[1],:]
    path = ["" for i in 1:length(D)]
    for t in 1:length(V)
        for x in 1:length(V)
            if max(D[t], D[x] + w[x, t]) != Inf
                if D[x] + w[x, t] >= D[t] && x!=t
                    path[t] = string(V[x])
                end
                D[t] = max(D[t], D[x] + w[x, t])
            else
                if D[t] >= D[x] + w[x, t] && x!=t
                    path[t] = string(V[x])
                end
                D[t] = min(D[t], D[x] + w[x, t])
            end
        end
        if D[t] == Inf
            path[t] = "None"
        else
            path[t] *= V[t]
        end
    end
    for t in 1:length(V)
        # In the vector path : For each node, we have his precedent node in the best path
        # Now we have to do the path in reverse for each node
        while path[t] != "None" && path[t][1] != s
            precendent = findall(x->x==path[t][1], V)[1] # the precedent node
            path[t] = path[precendent][1:length(path[precendent])-1]*path[t]
        end
    end
    return D, path
end

PlusLongChemin (generic function with 1 method)

In [7]:
# Test
w = [0 2 2 Inf Inf Inf;
     Inf 0 Inf 3 Inf Inf;
     Inf Inf 0 1 1 Inf;
     Inf Inf Inf 0 Inf 4;
     Inf Inf Inf Inf 0 1;
     Inf Inf Inf Inf Inf 0]
V = ['A','B','C','D','E','F']

f = PlusLongChemin(V, w, 'A')

([0.0, 2.0, 2.0, 5.0, 3.0, 9.0], ["A", "AB", "AC", "ABD", "ACE", "ABDF"])